In [1]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
10,application_1765289937462_0011,pyspark,idle,Link,Link,None,
11,application_1765289937462_0012,pyspark,idle,Link,Link,None,
14,application_1765289937462_0015,pyspark,idle,Link,Link,None,
15,application_1765289937462_0016,pyspark,idle,Link,Link,None,
16,application_1765289937462_0017,pyspark,idle,Link,Link,None,
17,application_1765289937462_0018,pyspark,busy,Link,Link,None,
18,application_1765289937462_0019,pyspark,idle,Link,Link,None,
19,application_1765289937462_0020,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession

from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col  #import needed functions

spark = SparkSession \
    .builder \
    .appName("DF query 1 execution") \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
20,application_1765289937462_0021,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df1 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv", header = True, inferSchema = True)
df2 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv", header = True, inferSchema = True)
df = df1.union(df2)
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(DR_NO=1307355, Date Rptd='2010 Feb 20 12:00:00 AM', DATE OCC='2010 Feb 20 12:00:00 AM', TIME OCC=1350, AREA=13, AREA NAME='Newton', Rpt Dist No=1385, Part 1-2=2, Crm Cd=900, Crm Cd Desc='VIOLATION OF COURT ORDER', Mocodes='0913 1814 2000', Vict Age=48, Vict Sex='M', Vict Descent='H', Premis Cd=501, Premis Desc='SINGLE FAMILY DWELLING', Weapon Used Cd=None, Weapon Desc=None, Status='AA', Status Desc='Adult Arrest', Crm Cd 1=900, Crm Cd 2=None, Crm Cd 3=None, Crm Cd 4=None, LOCATION='300 E  GAGE                         AV', Cross Street=None, LAT=33.9825, LON=-118.2695)

In [16]:
from pyspark.sql.functions import col, year, to_timestamp, count, sum, struct, collect_list, sort_array, round, row_number, desc, col, when
from pyspark.sql.window import Window
import time

start = time.time()

df_processed = df.withColumn("Year", year(to_timestamp(col("DATE OCC"), "yyyy MMM dd hh:mm:ss a")))
# Filter null values
df_filtered = df_processed.filter(col("Year").isNotNull() & col("Vict Descent").isNotNull())

df_grouped = df_filtered.groupBy("Year", "Vict Descent") \
    .agg(count("*").alias("count"))
# Count total per Year (for the percent)
df_yearly_totals = df_filtered.groupBy("Year").agg(count("*").alias("total_year"))
df_joined = df_grouped.join(df_yearly_totals, on="Year", how="inner")
df_with_percent = df_joined.withColumn("percentage", round((col("count") / col("total_year")) * 100, 1))

# window configuration: Per Year (partitionBy), order descending
windowSpec = Window.partitionBy("Year").orderBy(col("count").desc())

final_result = df_with_percent \
    .withColumn("rank", row_number().over(windowSpec)) \
    .filter(col("rank") <= 3) \
    .select(
        col("Year"), 
        col("Vict Descent"), 
        col("count").alias("#"), 
        col("percentage").alias("%")
    ) \
    .orderBy(col("Year").desc(), col("#").desc())

count_rows = final_result.count()
final_result.show(count_rows, truncate=False)
end = time.time()
print("Execution time:", end - start, "seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------+-----+----+
|Year|Vict Descent|#    |%   |
+----+------------+-----+----+
|2025|H           |34   |40.5|
|2025|X           |24   |28.6|
|2025|W           |13   |15.5|
|2024|H           |28576|29.1|
|2024|W           |22958|23.3|
|2024|X           |19984|20.3|
|2023|H           |69401|34.6|
|2023|W           |44615|22.2|
|2023|B           |30504|15.2|
|2022|H           |73111|35.6|
|2022|W           |46695|22.8|
|2022|B           |34634|16.9|
|2021|H           |63676|35.1|
|2021|W           |44523|24.5|
|2021|B           |30173|16.6|
|2020|H           |61606|35.3|
|2020|W           |42638|24.5|
|2020|B           |28785|16.5|
|2019|H           |72458|36.4|
|2019|W           |48863|24.5|
|2019|B           |33157|16.6|
|2018|H           |75958|36.4|
|2018|W           |52233|25.0|
|2018|B           |35340|16.9|
|2017|H           |78308|37.6|
|2017|W           |52744|25.3|
|2017|B           |34713|16.6|
|2016|H           |99135|38.7|
|2016|W           |63760|24.9|
|2016|B 

In [17]:
from pyspark.sql.functions import when
start = time.time()
final_result_named = final_result.withColumn("Vict Descent", 
    when(col("Vict Descent") == 'W', "White")
    .when(col("Vict Descent") == 'B', "Black")
    .when(col("Vict Descent") == 'H', "Hispanic/Latin/Mexican")
    .when(col("Vict Descent") == 'X', "Unknown"))

count_rows = final_result.count()
final_result_named.show(count_rows, truncate=False)
end = time.time()
print("Execution time:", end - start, "seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+-----+----+
|Year|Vict Descent          |#    |%   |
+----+----------------------+-----+----+
|2025|Hispanic/Latin/Mexican|34   |40.5|
|2025|Unknown               |24   |28.6|
|2025|White                 |13   |15.5|
|2024|Hispanic/Latin/Mexican|28576|29.1|
|2024|White                 |22958|23.3|
|2024|Unknown               |19984|20.3|
|2023|Hispanic/Latin/Mexican|69401|34.6|
|2023|White                 |44615|22.2|
|2023|Black                 |30504|15.2|
|2022|Hispanic/Latin/Mexican|73111|35.6|
|2022|White                 |46695|22.8|
|2022|Black                 |34634|16.9|
|2021|Hispanic/Latin/Mexican|63676|35.1|
|2021|White                 |44523|24.5|
|2021|Black                 |30173|16.6|
|2020|Hispanic/Latin/Mexican|61606|35.3|
|2020|White                 |42638|24.5|
|2020|Black                 |28785|16.5|
|2019|Hispanic/Latin/Mexican|72458|36.4|
|2019|White                 |48863|24.5|
|2019|Black                 |33157|16.6|
|2018|Hispanic/L